<a href="https://colab.research.google.com/github/alinnman/lineofsight/blob/main/lighthouse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple sample for line of sight
This sample calculates the line of sight from a lighthouse/mountain/skyscraper to two observers located at sea leavel, with different elevations above the sea surface. All **heights** are specified in meters. Calculations are made without and with refraction. For refraction **air pressure** (kPa), **temperature** (degrees Celsius) and **temperature gradient** (degrees Celsius / meter) must be entered. Default values are pre-set.

A calculation for an observer at sea level is also made, and a check is made to see if the parabola approximation ("8 inches per miles squared") is correct or not.

A calculation of obstructed height is also made, and presented as a separate table. 

Instructions for use: Fill in the parameters and **Press Ctrl-F9 or the arrow button below** to calculate the result which will be presented in the bottom cell. <br/>Allow some time for the first execution (Google Colab needs some time to start up).<br/>
When running for the first time you will get a warning about the code not originating from Google Colab (it is stored in GitHub). You can safely ignore this warning, but you may of course read the source code. Click "Show code" to inspect the source code (you may also modify). 

*Tip: To simulate severe refraction set the temperature gradient to 0.1 (i.e strong temperature inversion)*.

In [ ]:
from math import sqrt, pi, atan2, cos
from IPython.display import Markdown, display

# @markdown ### Enter parameters:
LIGHTHOUSE_HEIGHT = 56 # @param {"type":"integer"}
OBSERVER_HEIGHT_1 = 4 # @param {"type":"integer"}
OBSERVER_HEIGHT_2 = 40 # @param {"type":"integer"}
PRESSURE = 101 # @param {"type":"integer"}
TEMPERATURE = 20 # @param {"type":"integer"}
TEMP_GRADIENT = -0.01 # @param {"type":"number"}

EARTH_CIRCUMFERENCE_EQUATORIAL = 40075.017
EARTH_CIRCUMFERENCE_MERIDIONAL = 40007.86
EARTH_CIRCUMFERENCE = (EARTH_CIRCUMFERENCE_EQUATORIAL + EARTH_CIRCUMFERENCE_MERIDIONAL) / 2
EARTH_RADIUS = EARTH_CIRCUMFERENCE / (2 * pi)

R = EARTH_RADIUS*1000

def visibility_limit (RR : float, h1 : float, h2 : float, getObscurations : bool = False)\
    -> tuple [float, float, list[tuple[float,float]]]:
  ''' Geometry for line-of-sight '''
  x1a = sqrt (((RR + h1)**2) - RR**2)
  x1r = atan2 (x1a, RR)
  x1 = x1r * RR
  x2a = sqrt (((RR + h2)**2) - RR**2)
  x2r = atan2 (x2a, RR)
  x2 = x2r * RR

  obscurations = list()

  # Do split and get obscured height
  if getObscurations:
    for i in range (10 + 1):
      s = x1r * ((10 - i) / 10)
      ho = RR / cos (s) - RR
      obscurations.append ((x2 + s*RR, ho))

  return x1, x2, obscurations

def reportObsc (obsc : list[tuple[float,float]]) :
  display (Markdown ("Obscured heights (with refraction)"))
  tableString = "| Distance | Obstructed Height |\n"
  tableString += "| ----- | ----- |\n"

  for e in obsc:
    distance = e[0]
    distanceString = str(round(distance))
    obscHeight = e[1]

    if obscHeight == 0 and distance > 0:
      distanceString = distanceString + " and closer"
    tableString += "|" + distanceString + "|" + str(round(obscHeight,2)) + "|\n"
  display (Markdown(tableString))

DT_DH = TEMP_GRADIENT # Temperature shift with increasing elevation
K_FACTOR = 503*(PRESSURE*10)*(1/((TEMPERATURE+273)**2))*(0.0343 + DT_DH)
Ra = R / (1 - K_FACTOR) # This is the radius adjusted for refraction
# See https://en.wikipedia.org/wiki/Atmospheric_refraction#cite_note-Hirt2010-28

h1 = LIGHTHOUSE_HEIGHT # Height of lighthouse

display (Markdown ("# Results"))
display (Markdown ("HEIGHT of LIGHTHOUSE = " + str(h1) + " meters"))

for h2 in [0, OBSERVER_HEIGHT_1, OBSERVER_HEIGHT_2]: # Try some different observer elevations

  display (Markdown ("### CALCULATING for OBSERVER HEIGHT = " + str(h2) + " meters"))

  ## No refraction
 
  x1, x2, obsc = visibility_limit (R, h1, h2)
  distance = x1 + x2
  lightHouseTable = "|Refraction|Line of Sight (meters)|\n"
  lightHouseTable += "|-----|-----|\n"
  lightHouseTable += "|No|" + str(round(distance))+ "|\n"

  # For zero elevation (observer at sea level) we can double-check with the parabola approximation
  # NOTE: The parabola approximation is useless if observer is above sea level!
  if h2 == 0:
    outputStr = "*Checking the 8 inches per miles squared approximation*<br/>"
    MILES_PER_KM = 0.621371
    miles = (distance/1000) * MILES_PER_KM
    inches = 8 * (miles**2)
    INCHES_PER_METER = 39.3701
    meters_from_inches = inches / INCHES_PER_METER
    outputStr += "*Parabola approximation of lighthouse height = " + str(round(meters_from_inches,5)) + " meters*<br/>"
    outputStr += "*.. which is " + \
                 str(abs(round(((meters_from_inches - h1) / h1),6)*100)) + \
                 " % off the real value*"
    display (Markdown (outputStr))

  ## Now adjust for refraction

  x1,x2,obsc = visibility_limit (Ra, h1, h2, getObscurations=True)
  distance = x1 + x2

  lightHouseTable += "|Yes|" + str(round(distance))+ "|\n"
  display (Markdown(lightHouseTable))
  if obsc is not None:
    reportObsc (obsc)


# Results

HEIGHT of LIGHTHOUSE = 56 meters

### CALCULATING for OBSERVER HEIGHT = 0 meters

*Checking the 8 inches per miles squared approximation*<br/>*Parabola approximation of lighthouse height = 55.99769 meters*<br/>*.. which is 0.0041 % off the real value*

|Refraction|Line of Sight (meters)|
|-----|-----|
|No|26716|
|Yes|28873|


...

| Distance | Obscured Height |
| ----- | ----- |
|36589|56.0|
|33702|45.36|
|30815|35.84|
|27927|27.44|
|25040|20.16|
|22153|14.0|
|19266|8.96|
|16378|5.04|
|13491|2.24|
|10604|0.56|
|7717 and closer|0.0|


### CALCULATING for OBSERVER HEIGHT = 40 meters

|Refraction|Line of Sight (meters)|
|-----|-----|
|No|49295|
|Yes|53274|


Obscured heights (with refraction)

| Distance | Obscured Height |
| ----- | ----- |
|53274|56.0|
|50387|45.36|
|47500|35.84|
|44613|27.44|
|41725|20.16|
|38838|14.0|
|35951|8.96|
|33064|5.04|
|30176|2.24|
|27289|0.56|
|24402 and closer|0.0|
